**Import Libraries**

In [ ]:
# We import necessary libraries. Numpy for math stuff, matplotlib for plotting graphs,
# tensorflow to train our model, and tensorflow_datasets to load dataset.
# We also import ResNet50 from keras for using a pre-trained model.

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input

**Load Dataset Info**

In [ ]:
# Here, we are loading the EuroSAT dataset's info to understand what data we have.
# This will tell us about the dataset like how many classes, types of data, etc.
dataset = tfds.builder('eurosat')
info = dataset.info
print(info)

**Check Features**

In [ ]:
# This line checks what kind of features our dataset got, like images and their labels.
info.features

**Get Class Names**

In [ ]:
#We make a list of all class names in our dataset so we know what categories we're dealing with.
#loop through each class, convert number to string name, and add it to the list.
class_names = []
for i in range(info.features["label"].num_classes):
  class_names.append(info.features["label"].int2str(i))

class_names

**List Dataset Splits**

In [ ]:
# This shows us all different parts of our dataset, like training, validation, and test splits.
list(info.splits.keys())

eurosat has 10 classes with total of 27000 images under 1 split

**Load Data Splits**

In [ ]:
# Here, we load our dataset into three parts: training, validation, and testing.
# We split training data into 80%, validation into next 10%, and test into last 10%.
(train, val, test) = tfds.load("eurosat/rgb", split=["train[:80%]", "train[80%:90%]", "train[90%:]"])

**Look at a Data Point**

In [ ]:
# We take a single data point from train data to see what it looks like. This helps us understand our data better.
datapoint = next(iter(train))
datapoint

This is a dictionary with 3 keys

## **Show Some Images**

In [ ]:
# We define a function to show a few images from our dataset. This makes it easy to see what kind of images we're working with.
# Then, we show 9 images as examples.

def show_images(dataset, num_images=9):
    plt.figure(figsize=(10, 10))
    for i, (image, label) in enumerate(dataset.take(num_images)):
        plt.subplot(3, 3, i + 1)
        plt.imshow(image)
        plt.title(class_names[label.numpy()])
        plt.axis('off')
    plt.show()

show_images(train)


**Set Training Parameters**

In [ ]:
# We're setting up some numbers to control our training, like how many times to go through data (epochs),
# how many images to process at a time (batch size), and a buffer size for shuffling data.
NUM_EPOCHS = 20
BATCH_SIZE = 128
BUFFER_SIZE = 1000

IMAGE_SHAPE = [180, 180]
NUM_CLASSES = info.features["label"].num_classes

**Preparing Dataset for Training**

In [ ]:
# We loading the training part of dataset with extra info like names of classes.
# This help us to understand what kind of pictures we dealing with in training.
train_dataset, dataset_info = tfds.load('eurosat/rgb', split='train', with_info=True)

# We get the names of all classes from dataset info. This names help us know categories like forest, sea, etc.
class_names = dataset_info.features['label'].names

# Initialize a dictionary to count occurrences of each class
class_counts = {class_name:0 for class_name in class_names}

for example in tfds.as_numpy(train_dataset):
    class_counts[class_names[example['label']]] += 1

# Plotting class distribution
plt.figure(figsize=(10, 8))
plt.bar(range(len(class_counts)), list(class_counts.values()), tick_label=list(class_counts.keys()))
plt.xticks(rotation=45)
plt.xlabel('Class Names')
plt.ylabel('Number of Examples')
plt.title('Class Distribution in EuroSAT Dataset')
plt.show()


## **Data augmentation**

In [ ]:
# Sometimes we need a random number between 0 and 1, maybe to decide if we gonna flip an image or not during data augmentation.
tf.random.uniform(())

In [ ]:
@tf.function
def prepare_training_data(datapoint):
  input_image = tf.image.resize(datapoint["image"], IMAGE_SHAPE)

# We can use this random number in conditions. For example, if random number is more than 0.5, we might do some extra image processing. It makes our data augmentation more dynamic.
  if tf.random.uniform(()) > 0.5:
    input_image = tf.image.random_flip_left_right(input_image)
    input_image = tf.image.random_flip_up_down(input_image)
    input_image = tf.image.random_brightness(input_image, max_delta=0.3)
    input_image = tf.image.random_saturation(input_image, lower=0.75, upper=1.5)
    input_image = tf.image.random_contrast(input_image, lower=0.75, upper=1.5)

  input_image = preprocess_input(input_image)

  return input_image, datapoint["label"]

def prepare_validation_data(datapoint):
  input_image = tf.image.resize(datapoint["image"], IMAGE_SHAPE)
  input_image = preprocess_input(input_image)

  return input_image, datapoint["label"]

In [ ]:
# Here, we preparing our training and validation data. We use a function for each to make sure data is ready for model.
# 'num_parallel_calls' helps to speed up processing by doing multiple tasks at once.
train = train.map(prepare_training_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
validation = val.map(prepare_validation_data)

train_dataset = train.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
validation_dataset = validation.batch(BATCH_SIZE)

## **Visualization of image after preprocessing**

In [ ]:
# We start by creating a big space to show our images, setting the size so everything looks big.
plt.figure(figsize=(15, 15))
for i in range(9):
  ax = plt.subplot(3, 3, i+1)
  for datapoint in tfds.as_numpy(train_dataset.take(1)):
    plt.imshow(datapoint[0][0].astype('uint8'))
    plt.title(class_names[datapoint[1][0]])
    plt.axis("off")

plt.show()

Without using uint8

In [ ]:
plt.figure(figsize=(15, 15))
for i in range(4):
  ax = plt.subplot(2, 2, i+1)
  for datapoint in tfds.as_numpy(train_dataset.take(1)):
    plt.imshow(datapoint[0][0])
    plt.title(class_names[datapoint[1][0]])
    plt.axis("off")
plt.show()

## **Building model**

In [ ]:
# First, we setting up ResNet50 model. We telling it the size of images it should expect and to use pre-trained 'imagenet' weights.
# 'include_top=False' means we not using the last part of model, cause we going to add our own layers for our specific task.
resnet = ResNet50(input_shape=IMAGE_SHAPE+[3], weights='imagenet', include_top=False)

# Here, we go through each layer in ResNet model we just setup, and tell them not to change during training.
# This because they already learned a lot from 'imagenet', and we wanna keep that knowledge.
for layer in resnet.layers:
  layer.trainable = False

# Now, we start adding our own layers on top of ResNet to make it suitable for our task.
# 'GlobalAveragePooling2D' helps to reduce the dimensions but keeps important stuff.
x = tf.keras.layers.GlobalAveragePooling2D()(resnet.output)
# 'Flatten' makes our data into a single list to be easier to work with in next layers.
x = tf.keras.layers.Flatten()(x)
# 'Dense' is a normal layer where we can learn stuff, we use 512 units here and 'relu' to decide what to activate.
x = tf.keras.layers.Dense(512, activation='relu')(x)
# Last layer is also 'Dense' where we decide between our classes. 'softmax' makes sure output is in probabilities.
predicition = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = tf.keras.models.Model(inputs=resnet.input, outputs=predicition)

In [ ]:
# Now, we telling our model how to learn. This step is like setting rules for how model should improve itself.
# 'loss' is what tells model how wrong its guesses are. We use 'SparseCategoricalCrossentropy' which is good for when we have many classes.
# 'from_logits=False' means our model outputs are already like probabilities, so no need to convert them.
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam', # 'adam' is a smart way to change model to make it better. It adjusts itself as model learns.
    metrics=['accuracy'] # We wanna know how often model guesses right, so we check 'accuracy'.
)

In [ ]:
# We calculating how many steps to do in one epoch when training.
# Since we using 80% of data for training, we multiply total number of examples by 0.8.
# Then, we divide by 'BATCH_SIZE' to know how many batches of data we got. We use '//' to get whole number.
STEPS_PER_EPOCH = int(info.splits["train"].num_examples * 0.8)//BATCH_SIZE

# Similar way, we calculate steps for validation. But this time, we use 10% of data ('0.1').
# Again, divide by 'BATCH_SIZE' to get how many batches we need to go through during validation.
VALIDATION_STEPS = int(info.splits["train"].num_examples * 0.1)//BATCH_SIZE

In [ ]:
# Now, we start the actual training of model. We telling it to look at our training data and learn from it.
# 'epochs = NUM_EPOCHS' means how many times model will go through all the training data.
# 'steps_per_epoch' is how many batches of images it will look at in one go before it says one epoch is done.
# This is cause we might not want to use all data in one epoch, especially if we got lots of it.
history = model.fit(
    train_dataset,
    epochs = NUM_EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=validation_dataset,
    validation_steps=VALIDATION_STEPS
)

In [ ]:
import pandas as pd

# Now, we take our training history which got all sorts of numbers like accuracy and loss over time,
# and we turning it into a pandas DataFrame. This is like making it into a neat table.
pd.DataFrame(
    history.history # 'history.history' is where TensorFlow keeps track of how model did after each epoch.
).plot()

## **Evaluating results**

In [ ]:
# Now, we getting our test data ready. This is the data we gonna use to finally check how good our model is after all the training.
test_dataset = test.map(prepare_validation_data)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
# Here we telling our model to check how well it does on test data.
# It will give us back numbers telling how accurate it is and how much error (loss) it got.
model.evaluate(test_dataset)

In [ ]:
# We making a big space to show a bunch of test images and our model's guesses.
plt.figure(figsize=(15, 15))
for i, datapoint in enumerate(tfds.as_numpy(test.take(25))):
  ax = plt.subplot(5, 5, i+1)
  plt.imshow(datapoint["image"])
  image = tf.image.resize(datapoint["image"], IMAGE_SHAPE)
  image = preprocess_input(image)
  image = np.expand_dims(image, axis=0)

  # Now, we make our model predict what it thinks this image is.
  # If the model's guess (highest probability class) matches the true label, we show the name in green.
  # If it's wrong, we show the name in red.

  if datapoint["label"] == np.argmax(model.predict(image)):
    plt.title(class_names[np.argmax(model.predict(image))], color="green")
  else:
    plt.title(class_names[np.argmax(model.predict(image))], color="red")

  plt.axis("off")

plt.show()

In [ ]:
from tensorflow.keras.preprocessing import image

# This function helps us to figure out what picture is showing by using our trained model.
def classify_image(image_path, model, preprocess_input, class_names):
    """
    Classifies an input image using the provided model.

    Parameters:
    - image_path: Path to the image to be classified.
    - model: Trained model to use for prediction.
    - preprocess_input: Preprocessing function to apply to the image before prediction.
    - class_names: List of class names corresponding to model output.

    Returns:
    - Predicted class name.
    """
    from tensorflow.keras.preprocessing import image
    import numpy as np
    # First, we load the image so we can show it to you.
    img = image.load_img(image_path)
    plt.imshow(img)
    plt.title("Input Image")
    plt.axis('off')  # Hide the axis
    plt.show()

    # Now we make the picture the right size and do the magic preprocessing.
    img = image.load_img(image_path, target_size=(180, 180))  # Adjusted to match model's expected input dimensions
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    preprocessed_image = preprocess_input(img_array_expanded_dims)

    # Predict the class
    predictions = model.predict(preprocessed_image)
    predicted_class = class_names[np.argmax(predictions)]

    return predicted_class


In [ ]:
# First, we assuming you already got everything set up: the model, how to make pictures ready, and the list of things it can guess.

# Now, we gonna try out our model with some pictures to see what it thinks they are.

# We got a picture saved at '/content/verification image 1.jpg'. We wanna know what our model thinks it is.
image_path = '/content/verification image 1.jpg'
predicted_class = classify_image(image_path, model, preprocess_input, class_names)

print(f"The predicted class is: {predicted_class}")


image_path = '/content/verification image 2.jpg'
predicted_class = classify_image(image_path, model, preprocess_input, class_names)

print(f"The predicted class is: {predicted_class}")

image_path = '/content/verification image 3.jpg'
predicted_class = classify_image(image_path, model, preprocess_input, class_names)

print(f"The predicted class is: {predicted_class}")